**Imports + DataClass**

In [2]:
import time
from dataclasses import dataclass
from typing import Callable, List, Optional

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

from classiq import *
from classiq.applications.iqae.iqae import IQAE

In [3]:
@dataclass
class SweepResult:
    num_qubits: int
    grid_size: int
    var_estimate: float
    abs_error: float
    runtime_sec: float
    total_iqae_iterations: int
    num_iqae_calls: int

**Configuration**

In [ ]:

# -----------------------------
MU = 0.15
SIGMA = 0.20
CONF_LEVEL = 0.95
ALPHA_VAR = 1 - CONF_LEVEL   # 0.05

L = 4  # truncation window in sigmas (mean ± L*sigma)

# IQAE settings (held constant across num_qubits)
IQAE_EPSILON = 0.02          # target absolute error on probability
IQAE_ALPHA = 0.01            # failure prob (confidence = 99%)

# Bisection settings
TOLERANCE = ALPHA_VAR / 10
BISECTION_STEPS = 30

print("alpha_var =", ALPHA_VAR)
print("iqae epsilon =", IQAE_EPSILON, "iqae alpha =", IQAE_ALPHA)
print("tolerance =", TOLERANCE, "max bisection steps =", BISECTION_STEPS)


alpha_var = 0.050000000000000044
iqae epsilon = 0.02 iqae alpha = 0.01
tolerance = 0.0050000000000000044 max bisection steps = 30


**Helpers**

In [6]:
def theoretical_var_return(mu: float, sigma: float, alpha_var: float):
    # VaR in return units = alpha-quantile of returns
    return stats.norm.ppf(alpha_var, loc=mu, scale=sigma)


def build_grid_and_probs(mu: float, sigma: float, num_qubits: int):
    n = 2 ** num_qubits
    low = mu - L * sigma
    high = mu + L * sigma

    grid_points = np.linspace(low, high, n)
    pdf_vals = stats.norm.pdf(grid_points, loc=mu, scale=sigma)

    probs = (pdf_vals / np.sum(pdf_vals)).tolist()
    total_prob = sum(probs)

    print(f"num_qubits={num_qubits:2d} | grid={n:5d} | sum(probs)={total_prob:.12f}")
    assert np.isclose(total_prob, 1.0, atol=1e-12), "Probability mass does not sum to 1"

    return grid_points, probs


**State-Prep**

In [7]:
# Global index used inside the payoff comparator
GLOBAL_INDEX = 0


def make_state_prep(probs: List[float]):
    @qfunc
    def load_distribution(asset: QNum):
        # Prepare |psi> = sum_i sqrt(probs[i]) |i>
        inplace_prepare_state(probs, bound=0, target=asset)

    @qperm
    def payoff(asset: Const[QNum], ind: QBit):
        # Inclusive: mark "good" if asset <= GLOBAL_INDEX
        global GLOBAL_INDEX
        ind ^= asset <= GLOBAL_INDEX

    @qfunc(synthesize_separately=True)
    def state_preparation(asset: QNum, ind: QBit):
        load_distribution(asset)
        payoff(asset, ind)

    return state_preparation


**One IQAE call: estimate alpha(index) + runtime + IQAE internal iterations**

In [8]:
def calc_alpha_quantum(index: int, num_qubits: int, state_preparation: Callable):
    """
    Returns:
      est_alpha: estimated P(asset <= index)
      runtime: wall-clock time for iqae.run()
      iqae_iters: number of IQAE iterations used internally (len(iterations_data))
    """
    global GLOBAL_INDEX
    GLOBAL_INDEX = index

    iqae = IQAE(
        state_prep_op=state_preparation,
        problem_vars_size=num_qubits,
        constraints=Constraints(max_width=28),
        preferences=Preferences(machine_precision=num_qubits),
    )

    t0 = time.perf_counter()
    iqae_res = iqae.run(epsilon=IQAE_EPSILON, alpha=IQAE_ALPHA)
    t1 = time.perf_counter()

    est_alpha = float(iqae_res.estimation)
    iqae_iters = len(iqae_res.iterations_data)

    return est_alpha, (t1 - t0), iqae_iters


**Bisection search that accumulates IQAE time + IQAE iterations**

In [13]:
def update_index(index: int, required_alpha: float, alpha_v: float, step: int):
    if alpha_v < required_alpha:
        return index + step
    return index - step


def value_at_risk_bisection(
    required_alpha: float,
    start_index: int,
    alpha_func,
    grid_size: int
):
    """
    Returns:
      var_idx: final grid index
      total_time: sum of IQAE runtimes across all alpha evaluations
      total_iqae_iters: sum of IQAE internal iterations across all runs
      num_calls: number of times we called alpha_func (i.e., number of IQAE runs)
    """
    index = start_index
    step = max(1, index // 2)

    alpha_v, t, iters = alpha_func(index)
    total_time = t
    total_iqae_iters = iters
    num_calls = 1

    for _ in range(BISECTION_STEPS):
        if np.isclose(alpha_v, required_alpha, atol=TOLERANCE) or step <= 0:
            break

        index = update_index(index, required_alpha, alpha_v, step)
        index = max(0, min(grid_size - 1, index))  # clamp
        step //= 2

        alpha_v, t, iters = alpha_func(index)
        total_time += t
        total_iqae_iters += iters
        num_calls += 1

    return index, total_time, total_iqae_iters, num_calls


**Sweep Qubits from 5 - 12**

In [ ]:
def sweep_num_qubits(start_q: int, end_q: int) -> List[SweepResult]:
    results: List[SweepResult] = []
    var_theory = theoretical_var_return(MU, SIGMA, ALPHA_VAR)

    for num_qubits in range(start_q, end_q + 1):
        grid_points, probs = build_grid_and_probs(MU, SIGMA, num_qubits)
        state_preparation = make_state_prep(probs)

        grid_size = 2 ** num_qubits
        start_index = grid_size // 4  # same “quarter point” heuristic

        def alpha_func(idx: int):
            return calc_alpha_quantum(idx, num_qubits, state_preparation)

        var_idx, runtime_sec, total_iqae_iters, num_calls = value_at_risk_bisection(
            ALPHA_VAR, start_index, alpha_func, grid_size
        )

        var_est = float(grid_points[var_idx])
        abs_err = abs(var_est - var_theory)

        results.append(
            SweepResult(
                num_qubits=num_qubits,
                grid_size=grid_size,
                var_estimate=var_est,
                abs_error=abs_err,
                runtime_sec=runtime_sec,
                total_iqae_iterations=total_iqae_iters,
                num_iqae_calls=num_calls,
            )
        )

    return results


results = sweep_num_qubits(5, 12)

print("\nnum_qubits,grid_size,var_estimate,abs_error,runtime_sec,total_iqae_iterations,num_iqae_calls")
for r in results:
    print(
        f"{r.num_qubits},{r.grid_size},"
        f"{r.var_estimate:.10f},{r.abs_error:.10f},"
        f"{r.runtime_sec:.4f},{r.total_iqae_iterations},{r.num_iqae_calls}"
    )
    

**Plots**

In [ ]:
qs = [r.num_qubits for r in results]
abs_err = [r.abs_error for r in results]
var_est = [r.var_estimate for r in results]
var_theory = theoretical_var_return(MU, SIGMA, ALPHA_VAR)

fig, axes = plt.subplots(2, 1, figsize=(9, 10))

# Abs error (log scale)
axes[0].plot(qs, abs_err, marker="o", linewidth=2.0)
axes[0].set_yscale("log")
axes[0].set_title("Absolute Error vs num_qubits")
axes[0].set_xlabel("num_qubits")
axes[0].set_ylabel("abs error (log)")
axes[0].grid(True, which="both", alpha=0.25)

# VaR estimate vs theoretical
axes[1].plot(qs, var_est, marker="o", linewidth=2.0, label="Estimated VaR")
axes[1].axhline(var_theory, linestyle="--", linewidth=1.6, label="Theoretical VaR")
axes[1].set_title("VaR Estimate vs num_qubits")
axes[1].set_xlabel("num_qubits")
axes[1].set_ylabel("VaR estimate (return)")
axes[1].grid(True, alpha=0.25)
axes[1].legend(frameon=False)

plt.tight_layout()
plt.show()
